<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import argparse
from cmath import e
import copy
import glob
import math
import pwd
from tkinter import E
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from pathlib import Path
from torch.utils.data import DataLoader

In [10]:
class AEDataset():
  def __init__(self, X, y):
    self.X = torch.Tensor(X.values)
    self.y = torch.Tensor(y.values)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [11]:
#エンコーダの定義
class Encoder(nn.Module):
  def __init__(self, input_size, encoding_dim):
    super().__init__()
    self.fc1 = nn.Linear(input_size, encoding_dim[0])
    self.pool1 = nn.Dropout(0.5)
    self.fc2 = nn.Linear(encoding_dim[0], encoding_dim[1])
    self.pool2 = nn.Dropout(0.5)

  def forward(self, x):
    x = torch.tanh(self.fc1(x))
    x = self.pool1(x)
    x = torch.tanh(self.fc2(x))
    x = self.pool2(x)
    return x

class Decoder(nn.Module):
  def __init__(self, encoding_dim, input_size):
    super().__init__()
    self.fc3 = nn.Linear(encoding_dim[1], encoding_dim[0])
    self.fc4 = nn.Linear(encoding_dim[0], input_size)

  def forward(self, x):
    x = torch.tanh(self.fc3(x))
    x = torch.tanh(self.fc4(x))

class AutoEncoder(nn.Module):
  def __init__(self, input_size=13767, encoding_dim=[500,100]):
    super().__init__()
    self.encoder = Encoder(input_size, encoding_dim)
    self.decoder = Decoder(encoding_dim, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x